1. Import required libraries and components.

In [ ]:
import pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

2. Read the data set that you generated in the previous notebook.
For convenience, save the train and test subsets in CSV files.

In [ ]:
data = pd.read_csv("dataset.csv")

# Generate train and test splits
train_data, test_data = train_test_split(
    data,
    test_size=0.2,
    random_state=0
)

train_data.to_csv("train_subset.csv", index=False)
test_data.to_csv("test_subset.csv", index=False)

3. Define the exact same data engineering function as in the preceding notebook to apply the same transformations.

In [ ]:
def preprocess(df):
    variations = [df]
    for i in range(10000):
        variations.append(df.copy() * i)

    df_extended = pd.concat(variations, axis=1)

    return df_extended

4. Train the model by streaming the data from the file in chunks and preprocessing each chunk at once.
By doing so, only some of the data needs to fit into memory at the same time.

> NOTE You must use classifiers that work well with piecemeal, or "out-of-core", training.
The `GaussianNB` classifier, or Gaussian naive-Bayes, is one such classifier.

In [ ]:
model = GaussianNB()

batches = 0
for minibatch in pd.read_csv("train_subset.csv", chunksize=1000):
    X = minibatch.drop('target', axis=1)
    y = minibatch['target']
    X_processed = preprocess(X)
    model.partial_fit(X_processed, y, classes=y.unique())

    batches += 1
    print(f"Training batch {batches} done")

5. Test the trained model.

In [ ]:
test_data = pd.read_csv("test_subset.csv")
X_test = test_data.drop('target', axis=1)
y_test = test_data['target']


y_predicted = model.predict(preprocess(X_test))


print("Classification report:")
print(classification_report(y_test, y_predicted))